In [19]:
import pandas as pd
import numpy as np

import glob
import os

### 1. 지점명별 좌표데이터 호출

In [20]:
latlng_df = pd.read_csv('./data/지점명_좌표데이터.csv', encoding='cp949')
latlng_df = latlng_df.iloc[:,1:4]
latlng_df

,지점명,위도,경도
0,수서역사거리,37.487468,127.101325
1,신논현역 사거리,37.501817,127.025606
2,성수대교 남단 사거리,37.537047,127.034935
3,을지병원 앞 사거리,37.519680,127.028133
4,신사역 오거리,37.516287,127.020023
...,...,...,...
726,동1로지하차도,37.595102,127.080242
727,면목2동사거리,37.586626,127.079933
728,중랑구청사거리,37.606860,127.095233
729,면목2동사거리,37.586626,127.079933


In [21]:
# 지점명의 중간 공백 제거
for idx, val in latlng_df.iterrows():
    latlng_df['지점명'][idx] = latlng_df['지점명'][idx].replace(' ', '')

<ipython-input-21-8d5de4f69f56>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  latlng_df['지점명'][idx] = latlng_df['지점명'][idx].replace(' ', '')


In [22]:
latlng_df

,지점명,위도,경도
0,수서역사거리,37.487468,127.101325
1,신논현역사거리,37.501817,127.025606
2,성수대교남단사거리,37.537047,127.034935
3,을지병원앞사거리,37.519680,127.028133
4,신사역오거리,37.516287,127.020023
...,...,...,...
726,동1로지하차도,37.595102,127.080242
727,면목2동사거리,37.586626,127.079933
728,중랑구청사거리,37.606860,127.095233
729,면목2동사거리,37.586626,127.079933


### 2. 사고다발지역 rawdata (from TMACS)에 지점명, 위도, 경도, 자치구 칼럼 추가

In [23]:
# 자치구만 변경, 코드는 그대로
globbed_files = glob.glob("./사고다발지역_rawdata/종로구/*.xlsx")

In [24]:
globbed_files

['./사고다발지역_rawdata/종로구\\종로구_관철동14-1번지사거리.xlsx',
 './사고다발지역_rawdata/종로구\\종로구_광하문우체국사거리.xlsx',
 './사고다발지역_rawdata/종로구\\종로구_광화문앞삼거리.xlsx',
 './사고다발지역_rawdata/종로구\\종로구_동대문성곽공원앞사거리.xlsx',
 './사고다발지역_rawdata/종로구\\종로구_동대문역사거리.xlsx',
 './사고다발지역_rawdata/종로구\\종로구_동대문역삼거리.xlsx',
 './사고다발지역_rawdata/종로구\\종로구_동묘공원앞삼거리.xlsx',
 './사고다발지역_rawdata/종로구\\종로구_동묘앞역사거리.xlsx',
 './사고다발지역_rawdata/종로구\\종로구_서울전문학교앞사거리.xlsx',
 './사고다발지역_rawdata/종로구\\종로구_서울지방경찰청.xlsx',
 './사고다발지역_rawdata/종로구\\종로구_신설동역사거리.xlsx',
 './사고다발지역_rawdata/종로구\\종로구_안국역사거리.xlsx',
 './사고다발지역_rawdata/종로구\\종로구_이화사거리.xlsx',
 './사고다발지역_rawdata/종로구\\종로구_종로3가역사거리.xlsx',
 './사고다발지역_rawdata/종로구\\종로구_종로5가역사거리.xlsx',
 './사고다발지역_rawdata/종로구\\종로구_종로타워삼거리.xlsx',
 './사고다발지역_rawdata/종로구\\종로구_중부교육지원청부근사거리.xlsx',
 './사고다발지역_rawdata/종로구\\종로구_창신1동주민센터.xlsx',
 './사고다발지역_rawdata/종로구\\종로구_청계4가교차로.xlsx',
 './사고다발지역_rawdata/종로구\\종로구_청계6가앞.xlsx',
 './사고다발지역_rawdata/종로구\\종로구_트윈트리타워앞.xlsx',
 './사고다발지역_rawdata/종로구\\종로구_혜화동로터리.xlsx']

In [25]:
len(globbed_files)

22

In [26]:
import os

for file in globbed_files:
    frame = pd.read_excel(file)
    file_name = os.path.basename(file)     # os.path.basename(path) : 입력받은 경로의 기본 이름을 반환
    
    # 파일명에서 괄호 다음 부분은 제거
    if file_name.find('(') != -1:
        idx = file_name.find('(')
        file_name = file_name[:idx]
    
    # 파일명에서 'xlsx' 제거 (파일명에 괄호가 없어서 if문에 안걸리는 경우)
    file_name = file_name.replace('.xlsx', '')
    
    idx2 = file_name.find('_')
    
    # 파일명에서 자치구 주출 => file_gu
    file_gu = file_name[:idx2]
    
    # 파일명에서 지점명 추출 => file_name
    file_name = file_name[idx2 + 1:]  

    # frame 데이터프레임에 지점명, 위도, 경도, 자치구 컬럼 추가
    frame['지점명'] = file_name
    frame['지점명'][0] = '지점명'
    frame['지점명'][1] = ''
    
    # 지점명, 위도, 경도 데이터프레임에서 file_name(지점명)의 위도,경도 정보 가져오기
    find = latlng_df.loc[latlng_df['지점명'] == file_name]
    lat = find['위도'].values[0]
    lng = find['경도'].values[0]
    
    frame['위도'] = lat
    frame['위도'][0] = '위도'
    frame['위도'][1] = ''
    
    frame['경도'] = lng
    frame['경도'][0] = '경도'
    frame['경도'][1] = ''
    
    frame['자치구'] = file_gu
    frame['자치구'][0] = '자치구'
    frame['자치구'][1] = ''

    # 연령, 성별, 경력, 음주, 종별, 용도 칼럼 위치 조정
    frame['Unnamed: 11'][0] = '연령'
    frame['Unnamed: 12'][0] = '성별'
    frame['Unnamed: 13'][0] = '경력'
    frame['Unnamed: 14'][0] = '음주'
    frame['Unnamed: 15'][0] = '종별'
    frame['Unnamed: 16'][0] = '용도'
    
    # 필요없는 행, 열 삭제
    #frame = frame.drop(['Unnamed: 0'], axis=1)
    frame = frame.drop(['Unnamed: 25'], axis=1)
    frame = frame.drop(['사고구조분석표'], axis=1)
    frame = frame.drop(1)
    
    frame.to_csv('./사고다발지역_processing/종로구/%s_%s.csv' % (file_gu, file_name), encoding='utf-8-sig', header=False)

<ipython-input-26-d48b9c9274ce>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frame['위도'][0] = '위도'
D:\Anaconda\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
<ipython-input-26-d48b9c9274ce>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frame['경도'][0] = '경도'


In [27]:
# 확인
dd = pd.read_csv('./사고다발지역_processing/종로구/종로구_이화사거리.csv')
pd.set_option('display.max_columns', None)
dd

,0,년도,월,일,요일,시각,사고등급,사망자수,중상자수,경상자수,부상자수,연령,성별,경력,음주,종별,용도,관련자2,사고형태,기상상태,일광상태,사고유형1,사고유형2,법규위반,도로종류,지점명,위도,경도,자치구
0,2,2017,10,4,수요일,15,경상사고,0,0,2,0,45,남,10년미만,NaN,승용,비사업용,승용,진입회전사고,맑음,주간,차대차,측면직각충돌,안전거리미확보,특별광역시도,이화사거리,37.576117,127.002217,종로구
1,3,2018,8,7,화요일,0,경상사고,0,0,1,0,52,남,15년이상,NaN,승용,비사업용,승용,일직선사고,맑음,심야,차대차,추돌(진행중),안전운전불이행,특별광역시도,이화사거리,37.576117,127.002217,종로구
2,4,2018,11,15,목요일,0,경상사고,0,0,1,0,40,남,15년이상,NaN,이륜,기타,보행자,보행횡단사고,맑음,심야,차대사람,보도통행중,기타,특별광역시도,이화사거리,37.576117,127.002217,종로구
3,5,2019,9,25,수요일,15,중상사고,0,2,1,0,65,남,15년이상,NaN,승용,사업용,승용,일직선사고,맑음,주간,차대차,추돌(진행중),안전운전불이행,특별광역시도,이화사거리,37.576117,127.002217,종로구
4,6,2019,12,19,목요일,20,경상사고,0,0,1,0,40,남,15년이상,NaN,승합,사업용,승합,진입회전사고,맑음,출퇴근,차대차,측면직각충돌,안전거리미확보,특별광역시도,이화사거리,37.576117,127.002217,종로구
5,7,2017,3,18,토요일,13,경상사고,0,0,1,0,37,남,15년이상,NaN,승용,비사업용,승용,진입회전사고,맑음,주간,차대차,측면직각충돌,안전운전불이행,특별광역시도,이화사거리,37.576117,127.002217,종로구
6,8,2017,4,29,토요일,23,경상사고,0,0,1,0,43,남,15년이상,NaN,승용,비사업용,보행자,보행횡단사고,맑음,심야,차대사람,횡단중,보행자보호의무위반,특별광역시도,이화사거리,37.576117,127.002217,종로구
7,9,2017,5,16,화요일,18,부상사고,0,0,0,1,95,남,15년이상,NaN,자전거,기타,자전거,진입회전사고,맑음,출퇴근,차대차,측면직각충돌,신호위반,특별광역시도,이화사거리,37.576117,127.002217,종로구
8,10,2017,6,25,일요일,0,중상사고,0,1,0,0,75,남,15년이상,NaN,승용,사업용,보행자,보행횡단사고,흐림,심야,차대사람,횡단중,보행자보호의무위반,특별광역시도,이화사거리,37.576117,127.002217,종로구
9,11,2017,6,11,일요일,23,중상사고,0,1,0,0,26,남,5년미만,NaN,이륜,기타,이륜,진입회전사고,맑음,심야,차대차,측면직각충돌,신호위반,특별광역시도,이화사거리,37.576117,127.002217,종로구


### 3. 자치구 하나의 파일로 만들기 => '교통사고_00구.csv'

In [28]:
csv_files = glob.glob("./사고다발지역_processing/종로구/*.csv")
csv_files

['./사고다발지역_processing/종로구\\종로구_관철동14-1번지사거리.csv',
 './사고다발지역_processing/종로구\\종로구_광하문우체국사거리.csv',
 './사고다발지역_processing/종로구\\종로구_광화문앞삼거리.csv',
 './사고다발지역_processing/종로구\\종로구_동대문성곽공원앞사거리.csv',
 './사고다발지역_processing/종로구\\종로구_동대문역사거리.csv',
 './사고다발지역_processing/종로구\\종로구_동대문역삼거리.csv',
 './사고다발지역_processing/종로구\\종로구_동묘공원앞삼거리.csv',
 './사고다발지역_processing/종로구\\종로구_동묘앞역사거리.csv',
 './사고다발지역_processing/종로구\\종로구_서울전문학교앞사거리.csv',
 './사고다발지역_processing/종로구\\종로구_서울지방경찰청.csv',
 './사고다발지역_processing/종로구\\종로구_신설동역사거리.csv',
 './사고다발지역_processing/종로구\\종로구_안국역사거리.csv',
 './사고다발지역_processing/종로구\\종로구_이화사거리.csv',
 './사고다발지역_processing/종로구\\종로구_종로3가역사거리.csv',
 './사고다발지역_processing/종로구\\종로구_종로5가역사거리.csv',
 './사고다발지역_processing/종로구\\종로구_종로타워삼거리.csv',
 './사고다발지역_processing/종로구\\종로구_중부교육지원청부근사거리.csv',
 './사고다발지역_processing/종로구\\종로구_창신1동주민센터.csv',
 './사고다발지역_processing/종로구\\종로구_청계4가교차로.csv',
 './사고다발지역_processing/종로구\\종로구_청계6가앞.csv',
 './사고다발지역_processing/종로구\\종로구_트윈트리타워앞.csv',
 './사고다발지역_processing/종로구\\종로구_혜화동로터리

In [29]:
gu_data = pd.read_csv(csv_files[0])

for i in range(1, len(csv_files)):
    processed_data = pd.read_csv(csv_files[i])
    gu_data = pd.concat([gu_data, processed_data])

In [30]:
# 파일명만 변경해서 저장
gu_data.to_csv('./사고다발지역_after/교통사고_종로구.csv', encoding='utf-8-sig')

In [31]:
# 확인
testdf = pd.read_csv('./사고다발지역_after/교통사고_종로구.csv')
testdf

,Unnamed: 0,0,년도,월,일,요일,시각,사고등급,사망자수,중상자수,경상자수,부상자수,연령,성별,경력,음주,종별,용도,관련자2,사고형태,기상상태,일광상태,사고유형1,사고유형2,법규위반,도로종류,지점명,위도,경도,자치구
0,0,2,2017,7,13,목요일,9,부상사고,0,0,0,1,42,남,15년이상,NaN,화물,비사업용,원동기,진입회전사고,맑음,주간,차대차,측면직각충돌,안전거리미확보,특별광역시도,관철동14-1번지사거리,37.570176,126.983197,종로구
1,1,3,2017,9,1,금요일,3,중상사고,0,1,0,0,57,남,3년미만,NaN,승용,사업용,승합,기타사고,맑음,심야,차대차,차대차-기타,안전운전불이행,특별광역시도,관철동14-1번지사거리,37.570176,126.983197,종로구
2,2,4,2017,11,24,금요일,1,경상사고,0,0,1,0,56,남,15년이상,NaN,승용,사업용,보행자,진입회전사고,맑음,심야,차대사람,횡단중,신호위반,특별광역시도,관철동14-1번지사거리,37.570176,126.983197,종로구
3,3,5,2018,3,24,토요일,20,경상사고,0,0,1,1,30,남,2년미만,NaN,원동기,기타,보행자,기타사고,맑음,출퇴근,차대사람,차도통행중,안전운전불이행,특별광역시도,관철동14-1번지사거리,37.570176,126.983197,종로구
4,4,6,2018,7,28,토요일,18,경상사고,0,0,2,0,56,남,15년이상,NaN,승용,비사업용,승용,기타사고,맑음,출퇴근,차대차,차대차-기타,안전거리미확보,특별광역시도,관철동14-1번지사거리,37.570176,126.983197,종로구
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
838,76,78,2019,11,15,금요일,7,경상사고,0,0,1,0,58,남,15년이상,NaN,승용,비사업용,승용,기타사고,비,출퇴근,차대차,차대차-기타,안전거리미확보,특별광역시도,혜화동로터리,37.585533,127.001217,종로구
839,77,79,2019,11,7,목요일,8,경상사고,0,0,2,0,72,남,15년이상,NaN,화물,사업용,화물,진입회전사고,맑음,출퇴근,차대차,측면직각충돌,안전운전불이행,특별광역시도,혜화동로터리,37.585533,127.001217,종로구
840,78,80,2019,11,15,금요일,8,경상사고,0,0,1,0,66,남,15년이상,NaN,승용,사업용,승용,진입회전사고,맑음,출퇴근,차대차,측면직각충돌,안전거리미확보,특별광역시도,혜화동로터리,37.585533,127.001217,종로구
841,79,81,2019,12,10,화요일,6,경상사고,0,0,1,0,58,남,15년이상,NaN,승용,사업용,화물,진입회전사고,맑음,새벽,차대차,측면직각충돌,신호위반,특별광역시도,혜화동로터리,37.585533,127.001217,종로구
